In [11]:
import pandas as pd
from PIL import Image
from skimage import color

import numpy as np
# from colour import XYZ_to_CAM16
import numpy as np
# from colour.models import RGB_to_XYZ
# 读取图片
img = Image.open('image/ss.png')

# 获取图片尺寸
width, height = img.size

# 获取像素数据
pixels = list(img.getdata())

# 创建DataFrame,A代表透明度
try:
    df = pd.DataFrame(pixels, columns=['R', 'G', 'B','A'])
    df = df.drop(columns=['A'])
except:
    df = pd.DataFrame(pixels, columns=['R', 'G', 'B'])
# 添加X和Y列
df['X'] = df.index % width
df['Y'] = df.index // width

# 显示DataFrame
print(df)

          R    G    B    X    Y
0        34   96  147    0    0
1        39  101  148    1    0
2        28   92  130    2    0
3        37  102  130    3    0
4        20   87  104    4    0
...     ...  ...  ...  ...  ...
395995  108   67   45  490  799
395996  134   79   49  491  799
395997  160   84   52  492  799
395998  167   72   44  493  799
395999  172   60   46  494  799

[396000 rows x 5 columns]


In [12]:
from colorspacious import cspace_converter
RGB = np.array([df['R'], df['G'], df['B']]).T

converter = cspace_converter("sRGB255", "CIELCh")
# 将LCH颜色添加到DataFrame
lch = converter(RGB)
df['L'] = lch[:, 0]
df['C'] = lch[:, 1]
df['H'] = lch[:, 2]
df['L'] = df['L'].round()
df['C'] = df['C'].round()
df['H'] = df['H'].round()
df['count'] = 1
df


,R,G,B,X,Y,L,C,H,count
0,34,96,147,0,0,39.0,34.0,268.0,1
1,39,101,148,1,0,41.0,32.0,264.0,1
2,28,92,130,2,0,37.0,28.0,257.0,1
3,37,102,130,3,0,40.0,24.0,243.0,1
4,20,87,104,4,0,34.0,21.0,228.0,1
...,...,...,...,...,...,...,...,...,...
395995,108,67,45,490,799,33.0,26.0,54.0,1
395996,134,79,49,491,799,39.0,34.0,54.0,1
395997,160,84,52,492,799,44.0,43.0,49.0,1
395998,167,72,44,493,799,43.0,51.0,43.0,1


In [13]:
C_mean = df['C'].mean()
L_mean = df['L'].mean()
H_mean = df['H'].mean()

In [14]:
df_mean = df.groupby('H').agg({'R': 'mean', 'G': 'mean', 'B': 'mean','L': 'mean','C':'mean'}).reset_index()
df_mean['R'] = df_mean['R'].round().astype(int)
df_mean['G'] = df_mean['G'].round().astype(int)
df_mean['B'] = df_mean['B'].round().astype(int)
df_mean['L'] = df_mean['L'].round().astype(int)
df_mean['C'] = df_mean['C'].round().astype(int)
df_count = df.groupby([ 'H']).size().reset_index(name='v')
df_merged = pd.merge(df_mean, df_count, on='H')
df_merged

,H,R,G,B,L,C,v
0,0.0,100,64,76,31,16,1511
1,1.0,111,63,79,33,22,2032
2,2.0,124,64,85,36,27,1376
3,3.0,144,76,98,42,30,1359
4,4.0,143,87,104,44,24,2340
...,...,...,...,...,...,...,...
356,356.0,91,58,71,28,16,2481
357,357.0,78,42,56,22,17,2206
358,358.0,65,30,43,16,17,2419
359,359.0,104,60,76,30,20,1713


In [15]:
import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('H')

colors = ['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_merged_sorted['R'], df_merged_sorted['G'], df_merged_sorted['B'])]


fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图（按色相顺序）', showlegend=False)
fig.show()

In [16]:
fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none'))
fig.update_layout(title='色相分布图（按每个色相份额排序）', showlegend=False)
fig.show()

In [17]:
import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('L')

colors = ['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_merged_sorted['R'], df_merged_sorted['G'], df_merged_sorted['B'])]


fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图（按亮度排序）', showlegend=False)
fig.show()

In [18]:




import plotly.graph_objects as go

# Sort the DataFrame by 'H' column
df_merged_sorted = df_merged.sort_values('H')

# Calculate LCH values based on C_mean, L_mean, and H
lch_colors = [(L_mean, C_mean, h) for h in df_merged_sorted['H']]

# Convert LCH colors to RGB
converter = cspace_converter("CIELCh", "sRGB255")
rgb_colors = converter(lch_colors)

# Format RGB colors
colors = ['rgb({},{},{})'.format(int(r), int(g), int(b)) for r, g, b in rgb_colors]

fig = go.Figure(data=go.Pie(labels=df_merged_sorted['H'], values=df_merged_sorted['v'], marker=dict(colors=colors), textinfo='none', direction='clockwise', sort=False))
fig.update_layout(title='色相分布图(平均色度和亮度后)', showlegend=False)
fig.show()

In [19]:
df_learn = df.groupby(['C', 'H']).agg({ 'L': 'sum'}).reset_index()
df_learn.columns = ['C', 'H', 'L']
df_learn['C_36']= df_learn['C']*3.6

In [20]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=256, init='k-means++',algorithm='lloyd')
kmeans.fit(df_learn[['C_36', 'H',]], sample_weight=df_learn['L'])
# 获取聚类结果
df_predict = df.copy()
df_predict['C_36'] = df_predict['C']*3.6
labels = kmeans.predict(df_predict[['C_36', 'H']])
# 将聚类结果写回原始数据框中

In [21]:
df['Cluster'] = labels
df

,R,G,B,X,Y,L,C,H,count,Cluster
0,34,96,147,0,0,39.0,34.0,268.0,1,11
1,39,101,148,1,0,41.0,32.0,264.0,1,11
2,28,92,130,2,0,37.0,28.0,257.0,1,121
3,37,102,130,3,0,40.0,24.0,243.0,1,225
4,20,87,104,4,0,34.0,21.0,228.0,1,225
...,...,...,...,...,...,...,...,...,...,...
395995,108,67,45,490,799,33.0,26.0,54.0,1,168
395996,134,79,49,491,799,39.0,34.0,54.0,1,41
395997,160,84,52,492,799,44.0,43.0,49.0,1,91
395998,167,72,44,493,799,43.0,51.0,43.0,1,170


In [22]:
df_learn_show = df.groupby(['Cluster']).agg({ 'L': 'sum','C': 'mean','H': 'mean','R': 'mean', 'G': 'mean', 'B': 'mean','count': 'sum'}).reset_index()
df_learn_show['C'] = df_learn_show['C'].round().astype(int)
df_learn_show['H'] = df_learn_show['H'].round().astype(int)
df_learn_show['R'] = df_learn_show['R'].round().astype(int)
df_learn_show['G'] = df_learn_show['G'].round().astype(int)
df_learn_show['B'] = df_learn_show['B'].round().astype(int)

df_learn_show

,Cluster,L,C,H,R,G,B,count
0,0,87505.0,18,286,84,90,119,2269
1,1,77591.0,24,83,185,162,123,1151
2,2,33470.0,21,163,105,151,130,572
3,3,127345.0,45,128,118,159,82,2094
4,4,78798.0,16,16,146,109,112,1611
...,...,...,...,...,...,...,...,...
251,251,92791.0,31,140,91,133,88,1811
252,252,74675.0,9,19,73,54,55,3220
253,253,31948.0,3,356,54,48,50,1658
254,254,59962.0,24,192,94,165,161,950


In [23]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_show['C'],
    y=df_learn_show['H'],
    mode='markers',
    marker=dict(
        size=np.sqrt(df_learn_show['count'])/8,
        color=df_learn_show.apply(lambda row: f'rgb({row["R"]},{row["G"]},{row["B"]})', axis=1),
        opacity=0.8
    ),
    text=df_learn_show.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='对色彩进行聚类后的结果',
    xaxis=dict(title='C'),
    yaxis=dict(title='H'),
    showlegend=False
)

fig.show()


In [24]:
from sklearn.cluster import KMeans
kmeans2 = KMeans(n_clusters=5, init='k-means++',algorithm='lloyd')
kmeans2.fit(df_learn_show[['H','C']],sample_weight=df_learn_show['L'])
# 获取聚类结果
labels2 = kmeans2.predict(df_learn_show[[ 'H','C']])
df_learn_show['Cluster2'] = labels2
df_learn_show
# 将聚类结果写回原始数据框中 取出df_learn_show中cluster2相同的行中L值最大的行，用plotly绘制饼图，颜色用R，G，B值，大小用count值

,Cluster,L,C,H,R,G,B,count,Cluster2
0,0,87505.0,18,286,84,90,119,2269,1
1,1,77591.0,24,83,185,162,123,1151,3
2,2,33470.0,21,163,105,151,130,572,4
3,3,127345.0,45,128,118,159,82,2094,2
4,4,78798.0,16,16,146,109,112,1611,0
...,...,...,...,...,...,...,...,...,...
251,251,92791.0,31,140,91,133,88,1811,2
252,252,74675.0,9,19,73,54,55,3220,0
253,253,31948.0,3,356,54,48,50,1658,1
254,254,59962.0,24,192,94,165,161,950,4


In [25]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_show['C'],
    y=df_learn_show['H'],
    mode='markers',
    marker=dict(
        size=np.sqrt(df_learn_show['count'])/8,
        color=df_learn_show['Cluster2'],
        opacity=0.8
    ),
    text=df_learn_show.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='对色相聚一次类',
    xaxis=dict(title='C：色度'),
    yaxis=dict(title='H：色相'),
    showlegend=False
)

fig.show()

In [26]:
df_learn_show['final_v']=df_learn_show['C']*df_learn_show['L']

In [27]:
df_learn_l = df.groupby(['L']).agg({ 'count': 'sum'}).reset_index()

In [28]:
df_learn_show['L'] = (df_learn_show['L']/df_learn_show['count']).round().astype(int)

In [29]:
from sklearn.cluster import KMeans
kmeans3 = KMeans(n_clusters=10, init='k-means++',algorithm='lloyd')
kmeans3.fit(df_learn_l[['L']],sample_weight=df_learn_l['count'])
# 获取聚类结果
labels3 = kmeans3.predict(df_learn_l[[ 'L']])
df_learn_l['Cluster3'] = labels3
df_learn_l
# 将聚类结果写回原始数据框中 取出df_learn_show中cluster2相同的行中L值最大的行，用plotly绘制饼图，颜色用R，G，B值，大小用count值

,L,count,Cluster3
0,0.0,379,7
1,1.0,1053,7
2,2.0,1631,7
3,3.0,2992,7
4,4.0,9180,7
...,...,...,...
96,96.0,35,8
97,97.0,17,8
98,98.0,21,8
99,99.0,16,8


In [30]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_learn_l['L'],
    y=df_learn_l['count'],
    mode='markers',
    marker=dict(
        color=df_learn_l['Cluster3'],
        colorscale='Viridis',
        opacity=0.8
    )
))

fig.update_layout(
    title='在亮度上对像素聚类',
    xaxis=dict(title='L：亮度'),
    yaxis=dict(title='像素量'),
    showlegend=False
)

fig.show()


In [31]:
df_learn_l

,L,count,Cluster3
0,0.0,379,7
1,1.0,1053,7
2,2.0,1631,7
3,3.0,2992,7
4,4.0,9180,7
...,...,...,...
96,96.0,35,8
97,97.0,17,8
98,98.0,21,8
99,99.0,16,8


In [32]:
df_l_plan = df_learn_l.groupby('Cluster3').apply(lambda x: x.loc[x['count'].idxmax()]).reset_index(drop=True)
df_l_plan = df_l_plan.apply(lambda x: x.sort_values().reset_index(drop=True))
df_l_plan

/tmp/ipykernel_108005/210686816.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,L,count,Cluster3
0,5.0,2063.0,0.0
1,8.0,2332.0,1.0
2,25.0,3773.0,2.0
3,33.0,4821.0,3.0
4,42.0,5527.0,4.0
5,54.0,5620.0,5.0
6,62.0,8456.0,6.0
7,66.0,8494.0,7.0
8,70.0,11032.0,8.0
9,78.0,13645.0,9.0


In [33]:
df_color_plan = df_learn_show.groupby('Cluster2').apply(lambda x: x.loc[x['final_v'].idxmax()]).reset_index(drop=True)
df_color_plan

/tmp/ipykernel_108005/2810096399.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,Cluster,L,C,H,R,G,B,count,Cluster2,final_v
0,13.0,58.0,34.0,26.0,198.0,119.0,115.0,1978.0,0.0,3913842.0
1,11.0,38.0,33.0,274.0,46.0,91.0,142.0,3777.0,1.0,4703853.0
2,183.0,64.0,48.0,124.0,129.0,166.0,81.0,2081.0,2.0,6380112.0
3,82.0,52.0,67.0,45.0,209.0,84.0,43.0,2154.0,3.0,7494017.0
4,10.0,68.0,32.0,171.0,95.0,180.0,155.0,4127.0,4.0,8934016.0


In [34]:
df_plan = pd.DataFrame(columns=['L','C','H','R','G','B','count','count2','cluster2','cluster3'])

converter2 = cspace_converter("CIELCh","sRGB255")
# 将LCH颜色添加到DataFrame

lch = converter(RGB)
for idx, data in df_color_plan.iterrows():
    for idx2, data2 in df_l_plan.iterrows():
        L = data2['L']
        C = data['C']
        H = data['H']
        RGB = converter2((L, C, H))
        R = RGB[0]
        G = RGB[1]
        B = RGB[2]
        count = data['count']
        count2 = data2['count']
        cluster2 = data['Cluster2']
        cluster3 = data2['Cluster3']
        df_plan.loc[len(df_plan)] = [L,C,H,R,G,B,count,count2,cluster2,cluster3]

df_plan = df_plan.applymap(lambda x: 0 if x < 0 else x)
df_plan


/tmp/ipykernel_108005/1174046150.py:22: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



,L,C,H,R,G,B,count,count2,cluster2,cluster3
0,5.0,34.0,26.0,54.403510,0.000000,0.000000,1978.0,2063.0,0.0,0.0
1,8.0,34.0,26.0,60.600656,0.000000,0.000000,1978.0,2332.0,0.0,1.0
2,25.0,34.0,26.0,104.484576,36.643652,38.396790,1978.0,3773.0,0.0,2.0
3,33.0,34.0,26.0,126.237464,55.726629,55.645019,1978.0,4821.0,0.0,3.0
4,42.0,34.0,26.0,151.160989,77.531738,76.118258,1978.0,5527.0,0.0,4.0
5,54.0,34.0,26.0,185.127352,107.535599,104.911597,1978.0,5620.0,0.0,5.0
6,62.0,34.0,26.0,208.226721,128.167995,124.936665,1978.0,8456.0,0.0,6.0
7,66.0,34.0,26.0,219.909419,138.670066,135.175006,1978.0,8494.0,0.0,7.0
8,70.0,34.0,26.0,231.679113,149.293048,145.555439,1978.0,11032.0,0.0,8.0
9,78.0,34.0,26.0,255.474346,170.891369,166.720668,1978.0,13645.0,0.0,9.0


In [35]:
df_color_plan

,Cluster,L,C,H,R,G,B,count,Cluster2,final_v
0,13.0,58.0,34.0,26.0,198.0,119.0,115.0,1978.0,0.0,3913842.0
1,11.0,38.0,33.0,274.0,46.0,91.0,142.0,3777.0,1.0,4703853.0
2,183.0,64.0,48.0,124.0,129.0,166.0,81.0,2081.0,2.0,6380112.0
3,82.0,52.0,67.0,45.0,209.0,84.0,43.0,2154.0,3.0,7494017.0
4,10.0,68.0,32.0,171.0,95.0,180.0,155.0,4127.0,4.0,8934016.0


In [36]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=df_plan['cluster3'],
    y=df_plan['cluster2'],
    mode='markers',
    marker=dict(
        size=32,
        color=df_plan.apply(lambda row: f'rgb({row["R"]},{row["G"]},{row["B"]})', axis=1),
        opacity=0.8
    ),
    text=df_plan.apply(lambda row: f'R: {row["R"]}, G: {row["G"]}, B: {row["B"]}', axis=1)
))

fig.update_layout(
    title='配色方案',
    xaxis=dict(title='亮度'),
    yaxis=dict(title='颜色'),
    showlegend=False
)

fig.show()

In [37]:
import plotly.graph_objects as go

# Group the DataFrame by 'Cluster2' and select the row with the maximum 'L' value
df_max_l = df_learn_show.groupby('Cluster2').apply(lambda x: x.loc[x['final_v'].idxmax()]).reset_index(drop=True)

# Create the pie chart
fig = go.Figure(data=go.Pie(
    labels=df_max_l['Cluster2'],
    values=df_max_l['count'],
    marker=dict(
        colors=['rgb({},{},{})'.format(r, g, b) for r, g, b in zip(df_max_l['R'], df_max_l['G'], df_max_l['B'])]
    ),
    textinfo='none'
))

fig.update_layout(title='取色结果（如果图片颜色太少区别会很小，这不是bug是feature😡）', showlegend=False)
fig.show()


/tmp/ipykernel_108005/1545999088.py:4: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

